# 1) Scraping bio/characteristic data from Wikipedia

## 1.0) Setup

In [2]:
# Setup and parameters
import requests
import bs4

prefix="https://en.wikipedia.org/wiki/"

## 1.1 Define Scraping function
Function takes the following arguments:
-  Article List - A list of the articles you want to scrape from
-  Characteristics - The categories/characteristics to scrape for each article (coming from the bio/info box in the top right of the article)
-  Outfile - format for the output file, two files will be created 'filename.csv' with the collected data and 'filename_errors.log' that keeps track of the holes in the data 

In [3]:
def wiki_scrape(article_list=['My Chemical Romance'], char=['Genres'],outfile='wiki_scrape'):
    
    open(outfile+'.csv','w').close()
    open(outfile+'_errors.log','w').close()


    for article in article_list:
        char_count= [0]*len(char)
        suffix=article.replace(" ","_")
        url = prefix + suffix

        soup=bs4.BeautifulSoup(requests.get(url).text, 'lxml')
        
        # Find the info table
        table= soup.find('table')
        wiki_dict={}

        rows=table.find_all('tr')
        for row in rows:
            row_title=row.find('th')
            try:
                if row_title.text in char:
                    wiki_dict[row.find('th').text]=row.find('td').text
                    char_count[char.index(row_title.text)]=1

            except:
                pass

        with open(outfile+'.csv', "a") as wfile:
            article_output= '"'  + article + '","' + url + '","' + str(wiki_dict) + '"'
            wfile.write(str(article_output)+ "\n")

        if char_count != [1]*len(char):
            with open(outfile+'_errors.log', "a") as wfile:
                for count in char_count:
                    if count == 0:
                        wfile.write("Error: Couldn't find " + \
                                    "'" + char[char_count.index(count)] + "'"+ \
                                    " for " + "'" + article + "'" + "\n")

In [4]:
halftime=['Coldplay', 'Beyonce', 'Bruno Mars','Katy Perry', 'Lenny Kravitz' ,'Missy Elliott','Bruno Mars', 'Red Hot Chili Peppers']

In [5]:
wiki_scrape(halftime,['Born','Died','Genres', 'Children'])

In [7]:
import csv
newthing=[]
with open('wiki_scrape.csv','r') as f:
    stuff=csv.reader(f)
    for row in stuff:
        newthing.append(row)
dic=eval(newthing[0][2])
print(newthing[0][0],dic['Genres'])

Coldplay 


Alternative rock
pop rock
post-Britpop
pop





In [8]:
# Compare text allowing for mistakes
def raw(string):
    out=string.rstrip().lstrip().lower().replace('  ', ' ')
    return(out)

In [7]:
' mUse'.lstrip()

'mUse'

In [16]:
soup=bs4.BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/Muse_(disambiguation)").text,'lxml')

In [14]:
def is_disambig(soup):
    if soup.find_all('a',attrs={'title':'Help:Disambiguation'}) == []:
        return False
    else:
        return True

In [2]:
def is_article(soup):
    return

In [ ]:
# More features to build

### Page lookup right:
# Disambigution - if 'Disambigution pages' is in the bottom, then print ambiguity error
# Search result - select top result, if no character matches return special error?

# Just supply list of URLs (to reduce ambiguity)

# Formatting etc, save as dictionary rather than output, etc



# 2) Scrape Tables


In [50]:
from splinter import Browser

url = "https://en.wikipedia.org/wiki/President_of_Ireland"

browser = Browser('chrome', headless=False)
browser.visit(url)

In [55]:
url = "https://en.wikipedia.org/wiki/Nicki_Minaj_discography"
browser.visit(url)

## 2.1) Find the actual useful tables

In [63]:
# Find the actual useful tables (ones that start with 'wikitable')
tables = (tab for tab in browser.find_by_tag('table') if tab._element.get_attribute('class').split()[0] == 'wikitable')

# May not have a caption
for tab in tables:
    try:
        cap = tab.find_by_tag('caption').text
        print(cap)
    except:
        pass
    

List of studio albums, with selected chart positions, sales figures and certifications
List of reissued albums
List of compilation albums, with selected chart positions and certifications
List of mixtapes, with selected details
List of singles as lead artist, with selected chart positions and certifications, showing year released and album name
List of singles as featured artist, with selected chart positions and certifications, showing year released and album name
List of promotional singles, with selected chart positions and certifications, showing year released and album name
List of songs, with selected chart positions and certifications, showing year released and album name
List of non-single guest appearances, with other performing artists, showing year released and album name


In [52]:
data = []
rows = list(tab.find_by_tag('tr'))


for i, row in enumerate(rows):
    row_data = []
    
    # First row may be headers, slightly different tag
    if i == 0:
        
        if row.find_by_tag('th'):
            cells = row.find_by_tag('th')
            for cell in cells:
                row_data.append(cell.text)
                
            data.append(row_data)
            continue
        else:
            pass
            
    else:    
        cells = row.find_by_tag('td')

        for cell in cells:
            row_data.append(cell.text)

        data.append(row_data)
    
data

[]


In [79]:
def table_scrape(url):
        
    from splinter import Browser
    browser = Browser('chrome', headless=True)
    browser.visit(url)
    
    scraped_tables = []
    
    # Find the tables using find in splinter
    tables = (tab for tab in browser.find_by_tag('table') if tab._element.get_attribute('class').split()[0] == 'wikitable')
    
    for tab in tables:
        t_dict = {}
        
        # Not every table has a caption, if there is one capture it
        try:
            t_dict['caption'] = tab.find_by_tag('caption').text
        except:
            t_dict['caption'] = ""
        
        # Now the data
        data = []
        # Locate the rows of the table
        rows = list(tab.find_by_tag('tr'))

        for i, row in enumerate(rows):
            row_data = []

            # First row may be headers, slightly different tag
            if i == 0:

                # In which case we will append that data and continue on to next in iteration
                if row.find_by_tag('th'):
                    cells = row.find_by_tag('th')
                    for cell in cells:
                        row_data.append(cell.text)
                    data.append(row_data)
                    continue
                else:
                    pass
                
            # Regular case for all other rows (first row may fall into here if none header row)
            cells = row.find_by_tag('td')

            for cell in cells:
                row_data.append(cell.text)
            data.append(row_data)

        t_dict['data'] = data
        
        scraped_tables.append(t_dict)
        
    return(scraped_tables)



In [72]:
test = table_scrape("https://en.wikipedia.org/wiki/Nicki_Minaj_discography")

In [74]:
for tab in test:
    print(tab['caption'])

List of studio albums, with selected chart positions, sales figures and certifications
List of reissued albums
List of compilation albums, with selected chart positions and certifications
List of mixtapes, with selected details
List of singles as lead artist, with selected chart positions and certifications, showing year released and album name
List of singles as featured artist, with selected chart positions and certifications, showing year released and album name
List of promotional singles, with selected chart positions and certifications, showing year released and album name
List of songs, with selected chart positions and certifications, showing year released and album name
List of non-single guest appearances, with other performing artists, showing year released and album name


In [80]:
table_scrape("https://en.wikipedia.org/wiki/President_of_Ireland")

[{'caption': '',
  'data': [['No.',
    'Name\n(Birth–Death)',
    'Portrait',
    'Previous service',
    'Term of office',
    'Nominated by',
    'Election'],
   ['1',
    'Douglas Hyde\n(1860–1949)',
    '',
    'Senator\n(1925, 1938)',
    '25 June 1938',
    '24 June 1945',
    'All-party nomination',
    '1938'],
   ['2',
    "Seán T. O'Kelly\n(1882–1966)",
    '',
    'Tánaiste\n(1932–1945)',
    '25 June 1945',
    '24 June 1959',
    'Fianna Fáil',
    '1945'],
   ['Himself[n 3]', '1952'],
   ['3',
    'Éamon de Valera\n(1882–1975)',
    '',
    'Taoiseach\n(1932–1948, 1951–1954, 1957–1959)',
    '25 June 1959',
    '24 June 1973',
    'Fianna Fáil',
    '1959'],
   ['Fianna Fáil[n 4]', '1966'],
   ['4',
    'Erskine H. Childers\n(1905–1974)',
    '',
    'Tánaiste\n(1969–1973)',
    '25 June 1973',
    '17 November 1974',
    'Fianna Fáil',
    '1973'],
   ['5',
    'Cearbhall Ó Dálaigh[53]\n(1911–1978)',
    '',
    'Chief Justice of Ireland\n(1961–1973)',
    '19 December 